# Условия домашки

Вы работаете аналитиком в ветеринарной клинике SkyAnimal. Вас попросили провести  
сегментацию клиентских чеков по типу животного и его породе.

## Задача 1

Постройте группировку, по которой видно, какое количество заказов было для каждой  
пары животное - порода и какой средний чек был по этой паре

## Задача 2

Ответьте на следующие вопросы, используя функцию ttest_ind:  
- Какой средний чек выше - по собакам или по кошкам?
- Какой средний чек выше - по экзотическим животным или по собакам?
- Какой средний чек выше - по экзотическим животным или по кошкам?
- Существуют ли сегменты (разбиение по породе), в рамках которых средний чек по  
кошкам больше, чем по собакам?
- Какой средний чек выше - по экзотическим животным или по домашним породистым кошкам?

---

#### ***Установки и импорты файлов и библиотек***

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind
from matplotlib import pyplot as plt

In [2]:
file_name = 'Домашнее задание (исходник).xlsx'
df = pd.read_excel(file_name)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4846 entries, 0 to 4845
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id_order  4846 non-null   int64 
 1   animal    4846 non-null   object
 2   species   4846 non-null   object
 3   check     4846 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 151.6+ KB


In [3]:
df.sample(10)

,id_order,animal,species,check
3974,3647289,Кошка,Домашняя обыкновенная,4181
4785,3647607,Кошка,Уличная,2927
1404,3646281,Кошка,Домашняя породистая,17530
2627,3646773,Кошка,Домашняя породистая,42513
3640,3647157,Кошка,Домашняя обыкновенная,18026
658,3646003,Кошка,Уличная,2980
3604,3647146,Кошка,Домашняя обыкновенная,12826
3334,3647030,Экзотическое животное,Экзотическое животное,114016
4457,3647477,Собака,Домашняя породистая,10553
410,3645903,Собака,Домашняя обыкновенная,49013


#### ***Задача 1. Постройте группировку, по которой видно, какое количество заказов было для каждой пары животное - порода и какой средний чек был по этой паре***

In [4]:
print(df['animal'].unique())

['Кошка' 'Собака' 'Экзотическое животное']


In [5]:
print(df['species'].unique())

['Уличная' 'Домашняя обыкновенная' 'Домашняя породистая'
 'Экзотическое животное']


In [6]:
df_grb = df.groupby(['animal', 'species']).agg(avg_check = ('check', 'mean'), cnt_orders = ('id_order', 'count')).reset_index()
df_grb

,animal,species,avg_check,cnt_orders
0,Кошка,Домашняя обыкновенная,26252.648987,1037
1,Кошка,Домашняя породистая,27404.896000,375
2,Кошка,Уличная,24168.826087,598
3,Собака,Домашняя обыкновенная,26733.493478,920
4,Собака,Домашняя породистая,20443.963855,249
5,Собака,Уличная,25679.635659,774
6,Экзотическое животное,Экзотическое животное,28748.667413,893


---

#### ***Задача 2 Ответьте на вопросы, используя функцию ttest_ind***

- Какой средний чек выше - по собакам или по кошкам?
- Какой средний чек выше - по экзотическим животным или по собакам?
- Какой средний чек выше - по экзотическим животным или по кошкам?
- Существуют ли сегменты (разбиение по породе), в рамках которых средний чек по  кошкам больше, чем по собакам?
- Какой средний чек выше - по экзотическим животным или по домашним породистым кошкам?

In [7]:
def test_interpreter(statistic, pvalue, alpha=0.05):
    '''Функция комментирует результат теста  ttest_ind'''
    border_s = 1.96
    
    if (- border_s < statistic < border_s) and (pvalue >= alpha):
        print("Гипотеза Н0 подтверждается: средние равны")
        return True
    else:
        print("Гипотеза Н0 не подтверждается: средние не равны")
        return False
        
    
        
def inequality_means(df_1, df_2, name_parameter_df1=None, name_parameter_df2=None):
    '''Функция сравнивает средние величины'''
    df1_mean, df2_mean = df_1.mean()[0], df_2.mean()[0]
    if df1_mean > df2_mean:
        print(f'df_1{[name_parameter_df1]} больше чем df_2{[name_parameter_df2]} на {round(df1_mean - df2_mean, 2)}')
    if df1_mean < df2_mean:
        print(f'df_2[{name_parameter_df2}] больше df_1[{name_parameter_df1}] на {round(df2_mean - df1_mean, 2)}')


##### ***- Какой средний чек выше - по собакам или по кошкам?***

In [8]:
df1 = df.loc[df['animal'] == 'Собака', ['check']]
df2 = df.loc[df['animal'] == 'Кошка', ['check']]
s, p = ttest_ind(df1, df2)

if test_interpreter(s, p):
    print(f'{s = }')
    print(f'{p = }')
else:
    inequality_means(df1, df2, 'Собака', 'Кошка')

Гипотеза Н0 подтверждается: средние равны
s = array([-0.33249373])
p = array([0.73953411])


##### ***- Какой средний чек выше - по экзотическим животным или по собакам?***

In [9]:
df1 = df.loc[df['animal'] == 'Экзотическое животное', ['check']]
df2 = df.loc[df['animal'] == 'Собака', ['check']]
s, p = ttest_ind(df1, df2)

if test_interpreter(s, p):
    print(f'{s = }')
    print(f'{p = }')
else:
    inequality_means(df1, df2, 'Экзотическое животное', 'Собака')

Гипотеза Н0 не подтверждается: средние не равны
df_1['Экзотическое животное'] больше чем df_2['Собака'] на 3241.0


##### ***- Какой средний чек выше - по экзотическим животным или по кошкам?***

In [10]:
df1 = df.loc[df['animal'] == 'Экзотическое животное', ['check']]
df2 = df.loc[df['animal'] == 'Кошка', ['check']]
s, p = ttest_ind(df1, df2)

if test_interpreter(s, p):
    print(f'{s = }')
    print(f'{p = }')
else:
    inequality_means(df1, df2, 'Экзотическое животное', 'Кошка')

Гипотеза Н0 подтверждается: средние равны
s = array([1.90015591])
p = array([0.05751172])


##### ***- Существуют ли сегменты (разбиение по породе), в рамках которых средний чек по кошкам больше, чем по собакам?***


In [11]:
df1 = df.loc[df['animal'] == 'Собака']
df2 = df.loc[df['animal'] == 'Кошка']

for spcs in df['species'].unique():
    df11 = df1.loc[df1['species'] == spcs, ['check']]
    df21 = df2.loc[df2['species'] == spcs, ['check']]
    s, p = ttest_ind(df11, df21)
    
    print(spcs)
    if test_interpreter(s, p):
        print(f'{s = }')
        print(f'{p = }')
    else:
        inequality_means(df11, df21, 'Собака', 'Кошка')
    print()

Уличная
Гипотеза Н0 подтверждается: средние равны
s = array([0.85813042])
p = array([0.39097055])

Домашняя обыкновенная
Гипотеза Н0 подтверждается: средние равны
s = array([0.3322974])
p = array([0.73970032])

Домашняя породистая
Гипотеза Н0 не подтверждается: средние не равны
df_2[Кошка] больше df_1[Собака] на 6960.93

Экзотическое животное
Гипотеза Н0 не подтверждается: средние не равны



##### ***- Какой средний чек выше - по экзотическим животным или по домашним породистым кошкам?***

In [12]:
df1 = df.loc[df['species'] == 'Экзотическое животное', ['check']]
df2 = df.loc[(df['animal'] == 'Кошка') & (df['species'] == 'Домашняя породистая'), ['check']]
s, p = ttest_ind(df1, df2)

if test_interpreter(s, p):
    print(f'{s = }')
    print(f'{p = }')
else:
    inequality_means(df1, df2, 'Экзотическое животное', 'Кошка, Домашняя породистая')

Гипотеза Н0 подтверждается: средние равны
s = array([0.58378436])
p = array([0.55946939])
